In [3]:
import sys
sys.path.insert(0, '../')

import transformers
import torch.nn as nn
from transformers import AlbertModel, AlbertConfig
import torch
import argparse
import os
import easydict
from torch.utils.data import DataLoader, ConcatDataset
import pyxis.torch as pxt

from consonant.model.tokenization import NGRAMTokenizer
from consonant.model.modeling import Consonant


In [4]:
def load_tokenizer_model(ckpt):
    state = torch.load(ckpt)
    tokenizer = NGRAMTokenizer(state['ngram'])

    config = AlbertConfig(**state['config_dict'])
    model = Consonant(config)
    model.load_state_dict(state['model_state_dict'])
    return tokenizer, model

ckpt = '../output/comment_baseline_b390_savecheck/ckpt-0000100.bin'
tokenizer, model = load_tokenizer_model(ckpt)


In [5]:
def val_dataloader(args):
        
    # We should filter out only directory name excluding all the *.tar.gz files
    data_dir = os.path.join(args.pretrain_dataset_dir, 'val') 
    subset_list = [subset_dir for subset_dir in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, subset_dir))]
    train_dataset = ConcatDataset([pxt.TorchDataset(os.path.join(data_dir, subset_dir)) for subset_dir in subset_list])

    # Very small dataset for debugging
    # toy_dataset = Subset(train_dataset, range(0, 100)) # -> If you want to make 100sample toy dataset. 

    data_loader = DataLoader(
        train_dataset,
        batch_size=args.train_batch_size,
        num_workers=args.num_workers,
        pin_memory=True,
        shuffle=False
    )

    return data_loader

In [6]:
args = easydict.EasyDict({
    "pretrain_dataset_dir": '../dataset/processed/ratings_3_100',
    "train_batch_size": 128,
    "num_workers": 0,
})


In [9]:
valloader = val_dataloader(args)
model.cuda()

Consonant(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(17579, 128, padding_idx=0)
      (position_embeddings): Embedding(100, 128)
      (token_type_embeddings): Embedding(1, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=512, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=512, out_features=512, bias=True)
                (key): Linear(in_features=512, out_features=512, bias=True)
                (value): Linear(in_features=512, out_features=512, bias=True)
          

In [10]:
for batch in valloader:
    input_ids = batch['head_ids'].type(torch.LongTensor).cuda()
    answer_label = batch['midtail_ids'].type(torch.LongTensor).cuda()  
    attention_mask = batch['attention_masks'].type(torch.LongTensor).cuda()  
    
    #print(input_ids.shape, attention_mask.shape,  answer_label.shape)
    output = model(input_ids, attention_mask=attention_mask, token_type_ids=None, answer_label=answer_label)

    break

In [11]:
predict_label = output[1].argmax(dim=2)

In [13]:
print('===============')
for i in range(answer_label.shape[0]):
    predict_label[i][answer_label[i]==0]=0
    answer_string = tokenizer.decode_sent(input_ids[i].detach().cpu().numpy(), answer_label[i].detach().cpu().numpy())
    predict_string = tokenizer.decode_sent(input_ids[i].detach().cpu().numpy(), predict_label[i].detach().cpu().numpy())
    #print('===============')
    print('answer string\t:'+ answer_string)
    print('predict string\t:' + predict_string)
    print('===============')
    

answer string	:우아....새벽에보니까 무섭네 미스터리와에로스와 스릴러를 겸비한 영화!
predict string	:아아....사바아바니까 마사나 마사타리아아라사아 사리라라 가비하 아하!
answer string	:재미있음
predict string	:자마아아
answer string	:똥폼잡는것과 진짜 멋있는건 다르다 이것이 허세없는 멋이다.
predict string	:따파자나가가 자짜 마아나가 다라다 아가아 하사아나 마아다.
answer string	:그냥 챔피언십 나 봐라
predict string	:가나 차파아시 니 바라
answer string	:어이없었다..강리나가 아까워
predict string	:아아아아다..가리나가 아까아
answer string	:경쾌한 음악과 대비되는 날 것 그대로의 이야기,
predict string	:가카하 아아가 디바다나 나 가 가다라아 아아가,
answer string	:당시 손가락 터치는 명장면
predict string	:다사 사가라 타차나 마자마
answer string	:후반부를 느끼기에 초중반부는 너무 건조하잖아.
predict string	:하바바라 나까가아 차자바바나 나마 가자하자아.
answer string	:오늘 에서 하더라
predict string	:아나 아시 하다리
answer string	:빈 디젤표 허세 액션 보기 전부터 각본은 바라지 않았지만 액션만 봐도 그닥... 내 취향은 아님
predict string	:바 다자파 하사 아시 바가 자바타 가바아 바라자 아이자마 아사마 바다 기다... 나 차하아 아나
answer string	:검둥이 겨털 개혐오!!!!
predict string	:가다이 가타 가히아!!!!
answer string	:키예슬로프스키 감독 영화는 제목부터가 반은 먹고 들어간다
predict string	:카아사라파사카 기다 이하나 자마바타가 비아 마가 다아가다
answer string	:그런 여성은 세기 잘 나가는 커리어 우먼으로 살아야 하는 타입이다. 시대와